In [1]:
pip install spacy[transformers]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 671.3 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import json

#Load the Preamble Data for Training
preamble_data_file_ptr = open('/kaggle/input/lner-train-data/NER_TRAIN_PREAMBLE.json', 'r')
preamble_data = json.load(preamble_data_file_ptr)

In [6]:
#Load the Judgment Data for Training
judgement_data_file_ptr = open('/kaggle/input/lner-train-data/NER_TRAIN_JUDGEMENT.json', 'r')
judgement_data = json.load(judgement_data_file_ptr)

In [7]:
import random

#Merge and shuffle the data
data = preamble_data + judgement_data 
random.shuffle(data)
print(data[0])

{'id': '374811ce71a74625900b91f949b7411c', 'annotations': [{'result': [{'value': {'start': 12, 'end': 72, 'text': 'Anandi Mukta Sadguru Shree Mukta Jeevandasswami Suvarna Jaya', 'labels': ['PETITIONER']}, 'id': 'YBIHOR1Z', 'from_name': 'label', 'to_name': 'text', 'type': 'labels'}, {'value': {'start': 92, 'end': 103, 'text': 'V.R. Rudani', 'labels': ['RESPONDENT']}, 'id': 'ZI0QFZ9E', 'from_name': 'label', 'to_name': 'text', 'type': 'labels'}]}], 'data': {'text': 'Petitioner:\nAnandi Mukta Sadguru Shree Mukta Jeevandasswami Suvarna Jaya\n\n\tVs.\n\nRespondent:\nV.R. Rudani & Ors.\n\nDate Of Judgment'}, 'meta': {'source': 'civil_supremecourts preamble https://indiankanoon.org/doc/1728255/'}}


In [ ]:
data_labels = ['annotations', 'result', 'text', 'data', 'entities', 'value', 'labels', 'start', 'end']

In [10]:
training_data = []

#Iterate over the training data
for text in data:
    
    annotation_len = len(text[data_labels[0]])
    results_len = len(text[data_labels[0]][0][data_labels[1]])
    
    #If the data is present
    if annotation_len > 0 and results_len > 0:
        
        info_dict = {}
        info_dict[data_labels[2]] = text[data_labels[3]][data_labels[2]]
        info_dict[data_labels[4]] = []
        
        #Iterate over the results data and extract the start, end, and label values
        for data in text[data_labels[0]][0][data_labels[1]]:
            
            label = data[data_labels[5]][data_labels[6]][0]
            label_start = data[data_labels[5]][data_labels[7]]
            label_end = data[data_labels[5]][data_labels[8]]
            info_dict[data_labels[4]].append([label_start, label_end, label])
        
        training_data.append(info_dict)
        
print(training_data[0])

{'text': 'Petitioner:\nAnandi Mukta Sadguru Shree Mukta Jeevandasswami Suvarna Jaya\n\n\tVs.\n\nRespondent:\nV.R. Rudani & Ors.\n\nDate Of Judgment', 'entities': [[12, 72, 'PETITIONER'], [92, 103, 'RESPONDENT']]}


In [11]:
import spacy
from tqdm import tqdm
from spacy.tokens import DocBin

#Load a new spacy model
nlp = spacy.blank("en") 
#Create a DocBin object
doc_bin = DocBin()

In [54]:
from spacy.util import filter_spans

#Save the data in the form of DocBin object
for training_text in tqdm(training_data): 
    
    text = training_text[data_labels[2]]
    labels = training_text[data_labels[4]]
    doc = nlp.make_doc(text)
    entities = []
    
    for start, end, label in labels:
        span = doc.char_span(start, end, label = label, alignment_mode = "contract")
        if span is None:
            print("Skipping the entity")
        else:
            entities.append(span)
    
    filtered_entities = filter_spans(entities)
    doc.ents = filtered_entities
    doc_bin.add(doc)


doc_bin.to_disk("training_data.spacy") 

100%|██████████| 8816/8816 [00:04<00:00, 2203.71it/s]


In [55]:
!python -m spacy init fill-config /kaggle/input/lner-config-file/config_file.cfg config.cfg

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-11 22:40:40,851] [INFO] Set up nlp object from config
[2022-12-11 22:40:40,863] [INFO] Pipeline: ['transformer', 'ner']
[2022-12-11 22:40:40,868] [INFO] Created vocabulary
[2022-12-11 22:40:40,869] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias']
- This I

In [59]:
import json

#Load the Preamble Data for Testing
preamble_dev_data_file_ptr = open('/kaggle/input/lner-dev-data/NER_DEV_PREAMBLE.json', 'r')
preamble_dev_data = json.load(preamble_dev_data_file_ptr)

In [60]:
#Load the Judgment Data for Testing
judgement_dev_data_file_ptr = open('/kaggle/input/lner-dev-data/NER_DEV_JUDGEMENT.json', 'r')
judgement_dev_data = json.load(judgement_dev_data_file_ptr)

In [72]:
#Merge and shuffle the data
dev_data = preamble_dev_data + judgement_dev_data
random.shuffle(dev_data)

In [73]:
#Load the Model
nlp_ner = spacy.load("model-best")

#Predict the output by sending the text as input
doc = nlp_ner(dev_data[0][data_labels[3]][data_labels[2]])
colors = {'COURT': "#bbabf2", 'PETITIONER': "#f570ea", "RESPONDENT": "#cdee81", 'JUDGE': "#fdd8a5",
          "LAWYER": "#f9d380", 'WITNESS': "violet", "STATUTE": "#faea99", "PROVISION": "yellow",
          'CASE_NUMBER': "#fbb1cf", "PRECEDENT": "#fad6d6", 'DATE': "#b1ecf7", 'OTHER_PERSON': "#b0f6a2",
          'ORG':'#a57db5','GPE':'#7fdbd4'}

options = {"colors": colors} 

spacy.displacy.render(doc, style = "ent", options = options, jupyter = True)

In [75]:
doc = nlp_ner(dev_data[1][data_labels[3]][data_labels[2]])
spacy.displacy.render(doc, style = "ent", options = options, jupyter = True)

In [80]:
doc = nlp_ner(dev_data[7][data_labels[3]][data_labels[2]])
spacy.displacy.render(doc, style = "ent", options = options, jupyter = True)

In [63]:
#Calculate the accuracy of the model by keeping track of the total and correct predictions

correct_pred_cnt = 0
total_cnt = 0

for data in dev_data:
    
    total_cnt += 1
    
    model_input_text = data[data_labels[3]][data_labels[2]]
    model_pred = nlp_ner(model_input_text)
    
    pred_label_list = []
    for ent in model_pred.ents:
        pred_label_list.append(ent.label_)
        
    label_list = []
    for label in data[data_labels[0]][0][data_labels[1]]:
        label_list.append(label[data_labels[5]][data_labels[6]][0])
          
    if pred_label_list == label_list:
        correct_pred_cnt += 1

accuracy = (correct_pred_cnt / total_cnt) * 100

72.71880819366852


In [83]:
print(f'{accuracy:.2f}')

72.72
